In [ ]:
# -*- coding: utf-8 -*-
"""
일주기 생체리듬 예측 AI v0.16.1 (past_info 로직 개선)

[v0.16.1 변경 사항]:
- past_info 추출 로직 개선:
  - 기존: past_info 탐색 시, 5일 길이의 과거 구간에서 양 끝의 불완전한 하루 데이터를 버리고 계산하여 정보 손실 발생.
  - 개선:
    1. 탐색 영역 확장: 불완전한 첫날과 마지막 날을 보완하기 위해, 앞뒤의 lookback, current 구간 데이터를 임시로 참조하여 '완전한 하루'를 구성. 이를 통해 경계선 부근에서도 안정적인 생체 마커 계산이 가능해짐.
    2. 선택 영역 유지: 마커 탐색은 확장된 영역에서 수행하되, 최종 마커는 반드시 원래의 5일 '과거' 구간 내에서만 선택.

[v0.16.0 최종 아키텍처 주요 변경]:
1.  버퍼 및 정렬 (데이터 파이프라인):
    - create_tfrecords: 이제 각 데이터 샘플은 자정부터 시작하여 자정으로 끝나는
      더 긴 데이터 조각(약 8~9일)을 포함합니다. 이를 통해 모델이 항상 자정 기준의
      데이터를 사용할 수 있도록 보장.
    - 원래 샘플의 시작 시간 오프셋(offset) 정보도 함께 저장하여, 모델이 최종 보정
      곡선을 원래 시간축에 정확히 정렬하는 데 사용.

2.  슬라이딩 윈도우 보정 (모델 아키텍처):
    - IntegratedModel: 모델 내부에 tf.TensorArray를 사용한 루프를 구현하여,
      입력된 긴 데이터 조각 안에서 3일짜리 lookback 창을 하루씩 이동.
    - 매일 이동된 창의 데이터를 FourierCorrectionModel에 입력하여, 해당 날짜에만
      적용될 고유한 일일 보정 곡선을 생성.
    - 이 과정을 반복하여, 각 날짜가 항상 직전 3일의 최신 정보를 바탕으로 보정되도록
      하는 슬라이딩 윈도우 보정을 완벽하게 구현.

3.  일일 재조정 기준 궤도 (CPU Pre-processing):
    - 물리 모델(ODE)은 학습 루프에서 분리되어, CPU에서 미리 하루 단위의 불연속적인
      기준 궤도를 계산.

4.  End-to-End 학습 및 강화된 손실 함수:
    - 위의 모든 딥러닝 모델은 최종 예측 오차를 바탕으로 가중치가 한 번에 업데이트되는
      End-to-End 방식으로 학습.
    - 궤도의 연속성과 앵커 일치를 강제하는 손실 함수의 가중치를 높여, 물리적으로
      더 타당하고 안정적인 궤적 생성을 유도.
"""

# =============================================================================
# 0. 라이브러리 임포트 및 파이프라인 설정
# =============================================================================
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Concatenate, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm import tqdm
import argparse
import random
import math

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- 파이프라인 제어 설정 ---
OUTPUT_DIR = "final_biometric_pipeline_output_v0.16.1_PastInfoFix"

# --- 데이터 및 모델 설정 (PC 환경 최적화) ---
DATA_DURATION_DAYS = 30
INPUT_SEQUENCE_LENGTH = 7 * 24 * 60
PREDICTION_HORIZON = 60
PHASE_CORRECTION_LOOKBACK_DAYS = 3
DAY_MINUTES = 24 * 60
NUM_MARKERS_TO_KEEP = 5

TRAIN_RATIO = 0.8
VALIDATION_RATIO = 0.1

# --- 모델 하이퍼파라미터 (성능 튜닝) ---
D_MODEL = 192
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 384
DROPOUT_RATE = 0.05
NUM_FOURIER_HARMONICS = 5
LSTM_UNITS = 64

# --- 학습 하이퍼파라미터 (성능 튜닝) ---
EPOCHS = 15
BATCH_SIZE = 8
LEARNING_RATE = 0.0005
LAMBDA_REG = 0.1
LAMBDA_CONT = 10.0
LAMBDA_ANCHOR = 5.0

# --- 하이퍼파라미터 검증 ---
assert D_MODEL % NUM_HEADS == 0, f"D_MODEL({D_MODEL})은 NUM_HEADS({NUM_HEADS})로 나누어떨어져야 합니다."

# --- 물리 모델 파라미터 (St. Hilaire et al., 2007 기반) ---
PARAMS = {
    'mu': 0.13, 'q': 1/3, 'k': 0.55, 'alpha0': 0.1, 'I0': 9500,
    'p': 0.5, 'beta': 0.007, 'G': 37, 'rho': 0.032, 'tau_x': 24.2,
}

# =============================================================================
# 1. 생체리듬 물리 모델 (SCN 코어 엔진) 및 헬퍼 함수
# =============================================================================
def find_hr_nadir(heart_rate_data, is_sleeping_data, day_minutes=1440):
    num_days = len(heart_rate_data) // day_minutes
    daily_nadirs = []
    for day in range(num_days):
        day_start, day_end = day * day_minutes, (day + 1) * day_minutes
        day_hr, day_sleep = heart_rate_data[day_start:day_end], is_sleeping_data[day_start:day_end]
        sleep_hr = day_hr[day_sleep == 1]
        if len(sleep_hr) > 0:
            original_indices = np.where(day_sleep == 1)[0]
            daily_nadirs.append(original_indices[np.argmin(sleep_hr)])
        else:
            daily_nadirs.append(np.argmin(day_hr))
    return np.mean(daily_nadirs) if daily_nadirs else day_minutes / 2

def _sigmoid(x, k=2, x0=0):
    return 1 / (1 + np.exp(-k * (x - x0)))

def lco_model_ode(t, y, params, light_func, sleep_func):
    x, xc, n = y
    if not np.all(np.isfinite(y)): return [0,0,0]

    mu, q, k, alpha0, I0, p, beta, G, rho, tau_x = params.values()
    I, sigma = light_func(t), sleep_func(t)
    I = max(I, 0)
    alpha = alpha0 * ((I / I0)**p) * (I / (I + 100.0)) if I > 0 else 0
    B_hat = G * (1 - n) * alpha
    B = B_hat * (1 - 0.4 * x) * (1 - 0.4 * xc)

    cbt_min_phase_angle = -170.7 * np.pi / 180.0
    current_phase = np.arctan2(xc, x)
    phase_diff_rad = (current_phase - cbt_min_phase_angle + np.pi) % (2 * np.pi) - np.pi
    psi_c_x = phase_diff_rad * (tau_x / (2 * np.pi)) + (tau_x / 2)
    weight_enter = _sigmoid(psi_c_x, k=2, x0=16.5)
    weight_exit = 1 - _sigmoid(psi_c_x, k=2, x0=21.0)
    wmz_weight = weight_enter * weight_exit * sigma
    Ns_hat_normal = rho * (1/3.0 - sigma)
    Ns_hat_wmz = rho * (1/3.0)
    Ns_hat = Ns_hat_normal * (1 - wmz_weight) + Ns_hat_wmz * wmz_weight
    Ns = Ns_hat * (1 - np.tanh(10 * x))

    dxdt = (np.pi / 12.0) * (xc + mu * (x/3.0 + (4.0/3.0)*x**3 - (256.0/105.0)*x**7) + B + Ns)
    tau_term_sq = (24.0 / (0.99729 * tau_x))**2
    dxc_dt = (np.pi / 12.0) * (q * B * xc - x * (tau_term_sq + k * B))
    dn_dt = 60.0 * (alpha * (1 - n) - beta * n)
    return [dxdt, dxc_dt, dn_dt]

def lco_model_jacobian(t, y, params, light_func, sleep_func):
    x, xc, n = y
    if not np.all(np.isfinite(y)): return np.zeros((3,3))

    mu, q, k, alpha0, I0, p, beta, G, rho, tau_x = params.values()
    I, sigma = light_func(t), sleep_func(t)
    I = max(I, 0)
    alpha = alpha0 * ((I / I0)**p) * (I / (I + 100.0)) if I > 0 else 0
    cbt_min_phase_angle = -170.7 * np.pi / 180.0
    current_phase = np.arctan2(xc, x)
    phase_diff_rad = (current_phase - cbt_min_phase_angle + np.pi) % (2 * np.pi) - np.pi
    psi_c_x = phase_diff_rad * (tau_x / (2 * np.pi)) + (tau_x / 2)
    weight_enter = _sigmoid(psi_c_x, k=2, x0=16.5)
    weight_exit = 1 - _sigmoid(psi_c_x, k=2, x0=21.0)
    wmz_weight = weight_enter * weight_exit * sigma
    Ns_hat_normal = rho * (1/3.0 - sigma)
    Ns_hat_wmz = rho * (1/3.0)
    Ns_hat = Ns_hat_normal * (1 - wmz_weight) + Ns_hat_wmz * wmz_weight
    dB_dx = -0.4 * G * alpha * (1 - n) * (1 - 0.4 * xc)
    dB_dxc = -0.4 * G * alpha * (1 - n) * (1 - 0.4 * x)
    dB_dn = -G * alpha * (1 - 0.4 * x) * (1 - 0.4 * xc)
    dNs_dx = -Ns_hat * 10.0 * (1.0 / np.cosh(10 * x))**2
    J = np.zeros((3, 3))
    J[0, 0] = (np.pi / 12.0) * (mu * (1/3.0 + 4.0 * x**2 - (256.0*7.0/105.0) * x**6) + dB_dx + dNs_dx)
    J[0, 1] = (np.pi / 12.0) * (1.0 + dB_dxc)
    J[0, 2] = (np.pi / 12.0) * dB_dn
    B = G * alpha * (1 - n) * (1 - 0.4 * x) * (1 - 0.4 * xc)
    tau_term_sq = (24.0 / (0.99729 * tau_x))**2
    J[1, 0] = (np.pi / 12.0) * (q * xc * dB_dx - (tau_term_sq + k * B) - k * x * dB_dx)
    J[1, 1] = (np.pi / 12.0) * (q * B + q * xc * dB_dxc - k * x * dB_dxc)
    J[1, 2] = (np.pi / 12.0) * (q * xc * dB_dn - k * x * dB_dn)
    J[2, 2] = 60.0 * (-alpha - beta)
    return J

# =============================================================================
# 2. 딥러닝 모델 정의
# =============================================================================
@tf.keras.utils.register_keras_serializable()
class FourierTrajectoryLayer(tf.keras.layers.Layer):
    def __init__(self, num_harmonics, **kwargs):
        super(FourierTrajectoryLayer, self).__init__(**kwargs)
        self.num_harmonics = num_harmonics
        self.output_dim = DAY_MINUTES
        self.t = tf.linspace(0.0, 2 * np.pi, self.output_dim)
        self.t = tf.cast(self.t, dtype=tf.float32)

    def call(self, coeffs):
        coeffs_x = coeffs[:, :(1 + 2 * self.num_harmonics)]
        coeffs_xc = coeffs[:, (1 + 2 * self.num_harmonics):]
        traj_x = self._build_trajectory(coeffs_x)
        traj_xc = self._build_trajectory(coeffs_xc)
        return tf.stack([traj_x, traj_xc], axis=-1)

    def _build_trajectory(self, coeffs):
        a0 = coeffs[:, 0:1]
        a_n = coeffs[:, 1:self.num_harmonics + 1]
        b_n = coeffs[:, self.num_harmonics + 1:]
        trajectory = a0
        for n in range(1, self.num_harmonics + 1):
            trajectory += a_n[:, n-1:n] * tf.cos(n * self.t)
            trajectory += b_n[:, n-1:n] * tf.sin(n * self.t)
        return trajectory

    def get_config(self):
        config = super(FourierTrajectoryLayer, self).get_config()
        config.update({"num_harmonics": self.num_harmonics})
        return config

def build_fourier_correction_model(lookback_minutes, num_harmonics, lstm_units):
    num_coeffs_per_traj = 1 + 2 * num_harmonics
    output_size = num_coeffs_per_traj * 2

    input_lux = Input(shape=(lookback_minutes, 1), name='corr_input_lux')
    input_sleep = Input(shape=(lookback_minutes, 1), name='corr_input_sleep')
    input_body1 = Input(shape=(lookback_minutes, 3), name='corr_input_body1')
    input_body2 = Input(shape=(lookback_minutes, 1), name='corr_input_body2')
    input_zeit1 = Input(shape=(lookback_minutes, 1), name='corr_input_zeit1')
    input_zeit2 = Input(shape=(lookback_minutes, 1), name='corr_input_zeit2')
    input_zeit3 = Input(shape=(lookback_minutes, 1), name='corr_input_zeit3')

    def create_feat_extractor(inp, name):
        x = Conv1D(16, 30, activation='relu', padding='causal', name=f'corr_{name}_cnn1')(inp)
        x = Conv1D(8, 30, activation='relu', padding='causal', name=f'corr_{name}_cnn2')(x)
        return x

    features = [
        create_feat_extractor(input_lux, 'lux'), create_feat_extractor(input_sleep, 'sleep'),
        create_feat_extractor(input_body1, 'body1'), create_feat_extractor(input_body2, 'body2'),
        create_feat_extractor(input_zeit1, 'zeit1'), create_feat_extractor(input_zeit2, 'zeit2'),
        create_feat_extractor(input_zeit3, 'zeit3'),
    ]

    combined_feature_sequence = Concatenate(axis=-1)(features)
    lstm_output = LSTM(lstm_units, return_sequences=False, name='correction_lstm')(combined_feature_sequence)
    x = Dropout(0.2)(lstm_output)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    fourier_coeffs = Dense(output_size, activation='linear', name='fourier_coeffs')(x)

    model_inputs = [input_lux, input_sleep, input_body1, input_body2, input_zeit1, input_zeit2, input_zeit3]
    model = Model(inputs=model_inputs, outputs=fourier_coeffs, name='FourierCorrectionModel')
    return model

@tf.keras.utils.register_keras_serializable()
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.position = position
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(position, d_model)
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :], d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    def get_config(self):
        config = super(PositionalEncoding, self).get_config()
        config.update({"position": self.position, "d_model": self.d_model})
        return config

def build_main_feature_extractors(time_steps=INPUT_SEQUENCE_LENGTH, d_model=D_MODEL):
    feature_proportions = {
        'lco': 0.25, 'lux': 0.125, 'sleep': 0.125, 'body1': 0.25,
        'body2': 0.0625, 'zeit1': 0.0625, 'zeit2': 0.0625, 'zeit3': 0.0625,
    }
    cnn_block_map = { name: max(2, int(d_model * prop) // 2 * 2) for name, prop in feature_proportions.items() }
    current_sum = sum(cnn_block_map.values())
    if current_sum != d_model: cnn_block_map['lco'] += d_model - current_sum
    assert sum(cnn_block_map.values()) == d_model, "Sum of feature dimensions must equal d_model"

    def create_cnn_block(n_features, name_prefix):
        return tf.keras.Sequential([
            Conv1D(filters=32, kernel_size=5, activation='relu', padding='causal', name=f"{name_prefix}_cnn1"),
            Conv1D(filters=n_features, kernel_size=5, activation='relu', padding='causal', name=f"{name_prefix}_cnn2")
        ], name=f"{name_prefix}_cnn_block")

    input_lco = Input(shape=(time_steps, 2), name='input_lco')
    features_lco = create_cnn_block(cnn_block_map['lco'], 'lco')(input_lco)
    lco_feature_extractor = Model(inputs=input_lco, outputs=features_lco, name='LCOFeatureExtractor')

    input_lux = Input(shape=(time_steps, 1), name='input_lux')
    input_sleep = Input(shape=(time_steps, 1), name='input_sleep')
    input_body1 = Input(shape=(time_steps, 3), name='input_body1')
    input_body2 = Input(shape=(time_steps, 1), name='input_body2')
    input_zeit1 = Input(shape=(time_steps, 1), name='input_zeit1')
    input_zeit2 = Input(shape=(time_steps, 1), name='input_zeit2')
    input_zeit3 = Input(shape=(time_steps, 1), name='input_zeit3')
    features_lux = create_cnn_block(cnn_block_map['lux'], 'lux')(input_lux)
    features_sleep = create_cnn_block(cnn_block_map['sleep'], 'sleep')(input_sleep)
    features_body1 = create_cnn_block(cnn_block_map['body1'], 'body1')(input_body1)
    features_body2 = create_cnn_block(cnn_block_map['body2'], 'body2')(input_body2)
    features_zeit1 = create_cnn_block(cnn_block_map['zeit1'], 'zeit1')(input_zeit1)
    features_zeit2 = create_cnn_block(cnn_block_map['zeit2'], 'zeit2')(input_zeit2)
    features_zeit3 = create_cnn_block(cnn_block_map['zeit3'], 'zeit3')(input_zeit3)
    combined_other_features = Concatenate(axis=-1, name='combined_other_features')([
        features_lux, features_sleep, features_body1,
        features_body2, features_zeit1, features_zeit2, features_zeit3
    ])
    other_inputs = [input_lux, input_sleep, input_body1, input_body2, input_zeit1, input_zeit2, input_zeit3]
    other_feature_extractor = Model(inputs=other_inputs, outputs=combined_other_features, name='OtherFeatureExtractor')

    return lco_feature_extractor, other_feature_extractor

@tf.keras.utils.register_keras_serializable()
class ContextualTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
        super(ContextualTransformerBlock, self).__init__(**kwargs)
        self.d_model, self.num_heads, self.dff, self.rate = d_model, num_heads, dff, rate
        self.mha1 = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.mha2 = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([Dense(dff, activation='relu'), Dense(d_model)])
        self.layernorm1, self.layernorm2, self.layernorm3 = LayerNormalization(epsilon=1e-6), LayerNormalization(epsilon=1e-6), LayerNormalization(epsilon=1e-6)
        self.dropout1, self.dropout2, self.dropout3 = Dropout(rate), Dropout(rate), Dropout(rate)
    def call(self, inputs, training=False):
        past_info, current_info = inputs
        attn_output_current = self.mha1(query=current_info, key=current_info, value=current_info, training=training)
        current_info_sa = self.layernorm1(current_info + self.dropout1(attn_output_current, training=training))
        attn_output_cross = self.mha2(query=current_info_sa, key=past_info, value=past_info, training=training)
        current_info_contextualized = self.layernorm2(current_info_sa + self.dropout2(attn_output_cross, training=training))
        ffn_output = self.ffn(current_info_contextualized)
        final_output = self.layernorm3(current_info_contextualized + self.dropout3(ffn_output, training=training))
        return final_output
    def get_config(self):
        config = super(ContextualTransformerBlock, self).get_config()
        config.update({"d_model": self.d_model, "num_heads": self.num_heads, "dff": self.dff, "rate": self.rate})
        return config

@tf.keras.utils.register_keras_serializable()
class SelfAttentionBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
        super(SelfAttentionBlock, self).__init__(**kwargs)
        self.d_model, self.num_heads, self.dff, self.rate = d_model, num_heads, dff, rate
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([Dense(dff, activation='relu'), Dense(d_model)])
        self.layernorm1, self.layernorm2 = LayerNormalization(epsilon=1e-6), LayerNormalization(epsilon=1e-6)
        self.dropout1, self.dropout2 = Dropout(rate), Dropout(rate)
    def call(self, x, training=False):
        attn_output = self.mha(query=x, key=x, value=x, training=training)
        out1 = self.layernorm1(x + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))
    def get_config(self):
        config = super(SelfAttentionBlock, self).get_config()
        config.update({"d_model": self.d_model, "num_heads": self.num_heads, "dff": self.dff, "rate": self.rate})
        return config

@tf.keras.utils.register_keras_serializable()
class MainPredictor(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, prediction_horizon, rate=0.1, **kwargs):
        super(MainPredictor, self).__init__(**kwargs)
        self.num_layers, self.d_model, self.num_heads, self.dff, self.prediction_horizon, self.rate = num_layers, d_model, num_heads, dff, prediction_horizon, rate
        self.first_block = ContextualTransformerBlock(d_model, num_heads, dff, rate)
        self.other_blocks = [SelfAttentionBlock(d_model, num_heads, dff, rate) for _ in range(num_layers - 1)]
        self.prediction_head = tf.keras.Sequential([Dense(128, activation='relu', name="pred_head_dense1"), Dropout(rate), Dense(prediction_horizon, name="final_prediction")], name="PREDICTION_HEAD")

    def call(self, inputs, training=False):
        encoded_features, past_info, last_lco = inputs
        current_info_length = 2 * DAY_MINUTES
        current_info = encoded_features[:, -current_info_length:, :]
        x = self.first_block((past_info, current_info), training=training)
        for block in self.other_blocks: x = block(x, training=training)
        pooled_vector, last_vector = tf.reduce_mean(x, axis=1), x[:, -1, :]
        combined_final_vector = tf.concat([pooled_vector, last_vector, last_lco], axis=-1)
        predictions = self.prediction_head(combined_final_vector)
        return predictions

    def get_config(self):
        return {"num_layers": self.num_layers, "d_model": self.d_model, "num_heads": self.num_heads, "dff": self.dff, "prediction_horizon": self.prediction_horizon, "rate": self.rate}
    @classmethod
    def from_config(cls, config): return cls(**config)

@tf.keras.utils.register_keras_serializable()
class IntegratedModel(Model):
    def __init__(self, lco_feature_extractor, other_feature_extractor, fourier_correction_model, main_predictor, config, **kwargs):
        super().__init__(**kwargs)
        self.lco_feature_extractor = lco_feature_extractor
        self.other_feature_extractor = other_feature_extractor
        self.fourier_correction_model = fourier_correction_model
        self.main_predictor = main_predictor
        self.config = config
        self.fourier_layer = FourierTrajectoryLayer(config['num_harmonics'])
        self.pos_encoding_layer = PositionalEncoding(config['input_seq_len'], config['d_model'])
        self.lambda_reg, self.lambda_cont, self.lambda_anchor = config['lambda_reg'], config['lambda_cont'], config['lambda_anchor']

    def call(self, inputs, training=False):
        # 1. 입력 데이터 재구성
        time_offset = inputs['time_offset']
        batch_size = tf.shape(time_offset)[0]
        num_main_seq_days = self.config['input_seq_len'] // DAY_MINUTES

        # 2. 슬라이딩 윈도우를 이용한 일일 보정 곡선 생성 (tf.while_loop 사용)
        loop_len = num_main_seq_days + 1
        lookback_minutes = self.config['lookback_days'] * DAY_MINUTES

        # while_loop를 위한 초기값 설정
        i = tf.constant(0)
        correction_curves_ta = tf.TensorArray(tf.float32, size=loop_len, dynamic_size=False, clear_after_read=False, name="correction_curves_ta")

        # while_loop의 반복 조건: i < loop_len
        def cond(i, ta):
            return tf.less(i, loop_len)

        # while_loop의 반복 본문
        def body(i, ta):
            day_start_idx = i * DAY_MINUTES

            # 현재 날짜에 대한 lookback 데이터 슬라이싱
            lookback_data_list = []
            for key in ['corr_input_lux', 'corr_input_sleep', 'corr_input_body1', 'corr_input_body2', 'corr_input_zeit1', 'corr_input_zeit2', 'corr_input_zeit3']:
                lookback_data_list.append(inputs[key][:, day_start_idx : day_start_idx + lookback_minutes, :])

            # 푸리에 계수 예측 및 보정 곡선 생성
            fourier_coeffs = self.fourier_correction_model(lookback_data_list, training=training)
            daily_curve = self.fourier_layer(fourier_coeffs)

            # TensorArray에 결과 저장
            ta = ta.write(i, daily_curve)
            return i + 1, ta

        # tf.while_loop 실행
        _, final_ta = tf.while_loop(
            cond,
            body,
            loop_vars=[i, correction_curves_ta]
        )

        # TensorArray를 일반 텐서로 변환
        all_daily_curves = final_ta.stack()
        all_daily_curves_swapped = tf.transpose(all_daily_curves, [1, 0, 2, 3])

        # 3. 보정 곡선 및 모든 입력 데이터 정렬 (tf.gather 사용)
        full_correction_block = tf.reshape(all_daily_curves_swapped, [batch_size, -1, 2])
        input_seq_len = self.config['input_seq_len']

        # 각 샘플의 time_offset에 맞춰 슬라이싱할 인덱스를 한 번에 생성
        offsets = tf.cast(time_offset, dtype=tf.int32)
        sequence_indices = tf.range(input_seq_len, dtype=tf.int32)[tf.newaxis, :]
        time_indices = offsets + sequence_indices # 브로드캐스팅을 통해 (batch_size, input_seq_len) 모양의 인덱스 생성

        # tf.gather를 사용해 각 샘플에서 해당 인덱스의 데이터를 효율적으로 추출
        final_delta_trajectory = tf.gather(full_correction_block, time_indices, batch_dims=1)
        aligned_baseline = tf.gather(inputs['baseline_inputs'], time_indices, batch_dims=1)

        other_input_keys = ['input_lux', 'input_sleep', 'input_body1', 'input_body2', 'input_zeit1', 'input_zeit2', 'input_zeit3']
        main_cnn_inputs_other_list = [
            tf.gather(inputs[key], time_indices, batch_dims=1) for key in other_input_keys
        ]

        # 4. 최종 궤적 합성 및 특징 추출
        corrected_lco_trajectory = aligned_baseline + final_delta_trajectory

        lco_features = self.lco_feature_extractor(corrected_lco_trajectory, training=training)
        other_features = self.other_feature_extractor(main_cnn_inputs_other_list, training=training)
        combined_features = Concatenate(axis=-1)([lco_features, other_features])
        encoded_features = self.pos_encoding_layer(combined_features)

        # 5. Past Info 구성 및 최종 예측
        past_info_indices = inputs['past_info_indices']
        batch_indices = tf.tile(tf.range(batch_size, dtype=tf.int64)[:, tf.newaxis], [1, self.config['num_markers_to_keep'] * 3])
        gather_indices = tf.stack([tf.reshape(batch_indices, [-1]), tf.reshape(tf.maximum(past_info_indices, 0), [-1])], axis=1)
        past_info_flat = tf.gather_nd(encoded_features, gather_indices)
        past_info = tf.reshape(past_info_flat, [batch_size, self.config['num_markers_to_keep'] * 3, self.config['d_model']])
        past_info *= tf.cast(tf.not_equal(past_info_indices, -1), dtype=tf.float32)[:, :, tf.newaxis]

        last_lco_corrected = corrected_lco_trajectory[:, -1, :]
        y_pred = self.main_predictor((encoded_features, past_info, last_lco_corrected), training=training)

        # 6. 손실 계산
        if training:
            reg_loss = tf.reduce_mean(tf.square(all_daily_curves)) * self.lambda_reg
            self.add_loss(reg_loss)

            continuity_gaps = all_daily_curves_swapped[:, :-1, -1, :] - all_daily_curves_swapped[:, 1:, 0, :]
            cont_loss = tf.reduce_mean(tf.square(continuity_gaps)) * self.lambda_cont
            self.add_loss(cont_loss)

            last_day_corrected_x = tf.reshape(corrected_lco_trajectory, [batch_size, num_main_seq_days, DAY_MINUTES, 2])[:, -1, :, 0]
            predicted_cbt_nadir_idx = tf.cast(tf.argmin(last_day_corrected_x, axis=1), tf.float32)

            full_inputs_for_anchor = inputs['full_inputs_for_anchor']
            hr_series = full_inputs_for_anchor[:, 0]; sleep_series = full_inputs_for_anchor[:, 1]
            sleep_mask = tf.cast(sleep_series > 0.5, dtype=tf.float32)
            has_sleep = tf.reduce_any(tf.cast(sleep_mask, tf.bool), axis=1)
            hr_in_sleep = hr_series * sleep_mask + (1.0 - sleep_mask) * 1e9
            nadir_in_sleep_indices = tf.cast(tf.argmin(hr_in_sleep, axis=1), tf.float32)
            nadir_overall_indices = tf.cast(tf.argmin(hr_series, axis=1), tf.float32)
            actual_hr_nadir_idx = tf.where(has_sleep, nadir_in_sleep_indices, nadir_overall_indices)

            actual_cbt_nadir_idx = (actual_hr_nadir_idx + 120) % DAY_MINUTES
            anchor_loss_val = tf.reduce_mean(tf.square(predicted_cbt_nadir_idx - actual_cbt_nadir_idx)) / (DAY_MINUTES**2)
            anchor_loss = anchor_loss_val * self.lambda_anchor
            self.add_loss(anchor_loss)

        return y_pred

    def get_config(self): return {"config": self.config}
    @classmethod
    def from_config(cls, config_data, custom_objects=None): return cls(**config_data)

# =============================================================================
# 3. TFRecord 생성 및 파싱 (하이브리드 파이프라인)
# =============================================================================
def moving_average_np(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# [수정된 함수]
# past_info 추출 로직 개선: 경계선 데이터 손실을 최소화하는 새로운 로직 적용
def generate_past_info_indices(full_unscaled_df, main_seq_start_in_full_df, config):
    """
    개선된 past_info 인덱스 생성 함수.
    탐색 영역과 선택 영역을 분리하여 경계선 데이터의 손실을 최소화.

    Args:
        full_unscaled_df (pd.DataFrame): lookback + main_seq가 포함된 확장된 원본 데이터.
        main_seq_start_in_full_df (int): 확장된 데이터 내에서 main_seq가 시작되는 인덱스.
        config (dict): 모델 설정값.

    Returns:
        np.array: main_seq 기준의 상대적인 past_info 인덱스 배열 (15개).
    """
    num_markers_to_keep = config['num_markers_to_keep']
    day_minutes = config['day_minutes']
    padding_value = -1

    # 1. '선택 영역' 정의: 마커가 최종적으로 선택될 수 있는 원래의 '과거' 5일 구간.
    #    인덱스는 full_unscaled_df 기준의 절대 인덱스.
    selection_start_abs = main_seq_start_in_full_df
    # '과거' 구간은 7일 입력 시퀀스의 앞 5일.
    selection_end_abs = main_seq_start_in_full_df + (config['input_seq_len'] - 2 * day_minutes)

    # 2. '탐색 영역' 정의: 생체 마커 계산을 위해 사용되는 전체 데이터.
    #    경계 효과를 최소화하기 위해 전달받은 full_unscaled_df 전체를 사용.
    calculation_df = full_unscaled_df.assign(
        corrected_skin_temp=full_unscaled_df['skin_temp'] - full_unscaled_df['ambient_temp']
    )
    heart_rate = calculation_df['heart_rate'].values
    is_sleeping = calculation_df['is_sleeping'].values
    temp_series = calculation_df['corrected_skin_temp'].values

    # 3. 전체 '탐색 영역'에서 모든 잠재적 마커 후보 탐색
    # Nadir 마커 (수면 중 심박수 최저점)
    hr_in_sleep = np.where(is_sleeping > 0.5, heart_rate, np.inf)
    # 1시간 단위로 최저점을 찾아 여러 후보 확보
    nadir_candidates_abs = [
        i + np.argmin(hr_in_sleep[i:i+60])
        for i in range(0, len(calculation_df) - 60, 60)
        if np.any(np.isfinite(hr_in_sleep[i:i+60]))
    ]
    nadir_indices_abs = list(set(nadir_candidates_abs))

    # Onset/Offset 마커 (피부 온도 변화율 기반)
    if len(temp_series) > 40:
        # 이동 평균을 적용하여 노이즈 감소 후 변화율 계산
        temp_deriv_smoothed = moving_average_np(np.gradient(moving_average_np(temp_series, 30)), 10)
        offset_to_align = len(temp_series) - len(temp_deriv_smoothed)
        onset_indices_abs = list(np.where(temp_deriv_smoothed > 0.001)[0] + offset_to_align)
        offset_indices_abs = list(np.where(temp_deriv_smoothed < -0.001)[0] + offset_to_align)
    else:
        onset_indices_abs, offset_indices_abs = [], []

    # 4. 마커 필터링 및 선택
    #   - 1단계: '선택 영역' 내에 있는 마커만 필터링
    #   - 2단계: 최신순으로 정렬하여 상위 N개 선택
    final_indices_relative = []
    all_marker_candidates = [onset_indices_abs, nadir_indices_abs, offset_indices_abs]

    for marker_candidates in all_marker_candidates:
        # '선택 영역' (원래의 5일) 내에 있는 유효한 후보들만 필터링
        valid_candidates = [
            idx for idx in marker_candidates
            if selection_start_abs <= idx < selection_end_abs
        ]

        if len(valid_candidates) > 0:
            # '선택 영역'의 끝을 기준으로 가장 최신 마커 N개 선택
            top_indices_abs = sorted(valid_candidates, key=lambda idx: selection_end_abs - idx)[:num_markers_to_keep]
            # 절대 인덱스를 main_seq 기준의 상대 인덱스로 변환
            top_indices_relative = [idx - main_seq_start_in_full_df for idx in top_indices_abs]
            final_indices_relative.extend(top_indices_relative)
            # 필요 시 패딩 추가
            if len(top_indices_relative) < num_markers_to_keep:
                final_indices_relative.extend([padding_value] * (num_markers_to_keep - len(top_indices_relative)))
        else:
            # 유효한 마커가 없으면 패딩으로 채움
            final_indices_relative.extend([padding_value] * num_markers_to_keep)

    return np.array(final_indices_relative, dtype=np.int64)
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def _bytes_feature(value): return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def serialize_example(inputs_dict):
    feature = {key: _bytes_feature(tf.io.serialize_tensor(value).numpy()) for key, value in inputs_dict.items()}
    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

def create_tfrecords(df_unscaled, df_scaled, indices, config, file_path):
    print(f"--- TFRecord 파일 생성 시작: {file_path} ---")

    main_cnn_other_cols = { 'input_lux': ['lux'], 'input_sleep': ['is_sleeping'], 'input_body1': ['heart_rate', 'hrv', 'respiration_rate'], 'input_body2': ['skin_temp'], 'input_zeit1': ['meal_event'], 'input_zeit2': ['exercise_event'], 'input_zeit3': ['ambient_temp'], }
    fourier_corr_cols = { 'corr_input_lux': ['lux'], 'corr_input_sleep': ['is_sleeping'], 'corr_input_body1': ['heart_rate', 'hrv', 'respiration_rate'], 'corr_input_body2': ['skin_temp'], 'corr_input_zeit1': ['meal_event'], 'corr_input_zeit2': ['exercise_event'], 'corr_input_zeit3': ['ambient_temp'], }
    anchor_loss_cols = ['heart_rate', 'is_sleeping']

    num_main_seq_days = config['input_seq_len'] // DAY_MINUTES

    with tf.io.TFRecordWriter(file_path) as writer:
        for main_seq_start_idx in tqdm(indices, desc=f"{os.path.basename(file_path)} 생성 중"):
            # 1. 동적 버퍼링: 자정 기준으로 데이터 정렬
            time_offset = main_seq_start_idx % DAY_MINUTES

            # 자정 기준의 전체 데이터 슬라이스 시작/종료 인덱스 계산
            slice_start_midnight = main_seq_start_idx - time_offset
            total_days_needed = config['lookback_days'] + num_main_seq_days
            slice_end_midnight = slice_start_midnight + total_days_needed * DAY_MINUTES

            # 예측에 필요한 추가 데이터 길이
            final_slice_end = main_seq_start_idx + config['input_seq_len'] + config['pred_horizon']

            if slice_end_midnight > len(df_unscaled) or final_slice_end > len(df_unscaled):
                continue

            # [수정] past_info 계산을 위해 더 긴 unscaled slice를 사용
            df_slice_unscaled = df_unscaled.iloc[slice_start_midnight:final_slice_end]
            df_slice_scaled = df_scaled.iloc[slice_start_midnight:slice_end_midnight]
            len_past_info_calc = slice_end_midnight - slice_start_midnight
            past_info_calc_df = df_slice_unscaled.iloc[:len_past_info_calc]

            # 2. 데이터 준비
            example_dict = {}

            # 보정 모델용 (자정 기준, lookback + main_seq 길이)
            for key, cols in fourier_corr_cols.items():
                example_dict[key] = tf.constant(df_slice_scaled[cols].values, dtype=tf.float32)

            # 메인 모델용 (자정 기준, lookback + main_seq 길이)
            for key, cols in main_cnn_other_cols.items():
                example_dict[key] = tf.constant(df_slice_scaled[cols].values, dtype=tf.float32)

            example_dict['baseline_inputs'] = tf.constant(df_slice_scaled[['x_base', 'xc_base']].values, dtype=tf.float32)

            # 시간 오프셋 저장
            example_dict['time_offset'] = tf.constant([time_offset], dtype=tf.int32)

            # 앵커 손실용 (원래 시간축 기준 마지막 날)
            anchor_start = main_seq_start_idx + config['input_seq_len'] - DAY_MINUTES
            anchor_end = main_seq_start_idx + config['input_seq_len']
            example_dict['full_inputs_for_anchor'] = tf.constant(df_unscaled.iloc[anchor_start:anchor_end][anchor_loss_cols].values, dtype=tf.float32)

            # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
            # [수정된 호출] 개선된 함수를 사용하여 past_info 인덱스 생성
            # main_seq_start_idx가 아닌, 자정 기준 slice 내에서의 시작점(time_offset)을 전달
            example_dict['past_info_indices'] = generate_past_info_indices(
                full_unscaled_df=past_info_calc_df,
                main_seq_start_in_full_df=time_offset,
                config=config
            )
            # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

            # 정답 (원래 시간축 기준)
            label_start = main_seq_start_idx + config['input_seq_len']
            label_end = label_start + config['pred_horizon']
            example_dict['label'] = tf.constant(df_unscaled.iloc[label_start:label_end]['heart_rate'].values, dtype=tf.float32)

            writer.write(serialize_example(example_dict))
    print(f"--- TFRecord 파일 생성 완료: {file_path} ---")

# =============================================================================
# 4. 데이터 생성 함수 (현실적 자극 모델링)
# =============================================================================
def generate_realistic_biometric_data(output_dir, duration_days, person_profile):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    total_minutes = duration_days * DAY_MINUTES
    timestamps = pd.to_datetime('2025-07-01') + pd.to_timedelta(np.arange(total_minutes), 'm')
    df = pd.DataFrame(index=timestamps)

    event_cols = ['is_sleeping', 'meal_event', 'exercise_event', 'stress_event']
    for col in event_cols: df[col] = 0.0
    df['lux'] = 0.0

    for day in range(duration_days):
        day_start_idx, day_end_idx = day * DAY_MINUTES, (day + 1) * DAY_MINUTES
        daily_peak_lux = np.random.uniform(1000, 2500)
        minute_of_day = np.arange(DAY_MINUTES)
        daylight_start_min, daylight_end_min = 6 * 60, 20 * 60
        daylight_duration = daylight_end_min - daylight_start_min
        daylight_minutes = (minute_of_day - daylight_start_min)
        daylight_intensity = daily_peak_lux * np.sin(np.pi * daylight_minutes / daylight_duration)
        night_light = np.random.uniform(5, 20, DAY_MINUTES)
        is_daytime = (minute_of_day >= daylight_start_min) & (minute_of_day <= daylight_end_min)
        daily_lux = np.where(is_daytime, np.maximum(night_light, daylight_intensity), night_light)
        df.iloc[day_start_idx:day_end_idx, df.columns.get_loc('lux')] = daily_lux.clip(min=1)

    for day in range(duration_days):
        day_start_idx, current_day_ts = day * DAY_MINUTES, timestamps[day * DAY_MINUTES]
        is_weekend = current_day_ts.dayofweek >= 5

        if np.random.rand() < 0.5:
            activity_start = day_start_idx + np.random.randint(9 * 60, 17 * 60)
            activity_duration = np.random.randint(60, 180)
            activity_end = min(activity_start + activity_duration, total_minutes)
            df.iloc[activity_start:activity_end, df.columns.get_loc('lux')] += np.random.uniform(200, 500)

        sunrise_effect = np.random.normal(0, 0.25)
        wake_up_hour = np.random.normal(person_profile[f'wake_up_{"weekend" if is_weekend else "weekday"}_mean'],
                                        person_profile[f'wake_up_{"weekend" if is_weekend else "weekday"}_std']) - sunrise_effect
        sleep_duration = np.random.normal(person_profile['sleep_duration_mean'], 1.0 if is_weekend else 0.5)
        wake_up_minute = int(np.clip(wake_up_hour * 60, 0, DAY_MINUTES - 1))
        sleep_end_abs = day_start_idx + wake_up_minute
        sleep_start_abs = sleep_end_abs - int(sleep_duration * 60)
        df.iloc[max(0, sleep_start_abs) : min(total_minutes, sleep_end_abs), df.columns.get_loc('is_sleeping')] = 1

        meal_times = [wake_up_minute + np.random.randint(90, 180), int(np.random.normal(19, 1.0) * 60)] if is_weekend else [wake_up_minute + np.random.randint(30, 60), int(np.random.normal(12.5, 0.5) * 60), int(np.random.normal(19.5, 0.75) * 60)]
        for meal_time in meal_times:
            event_start = day_start_idx + meal_time
            if event_start < total_minutes: df.iloc[event_start : min(event_start + 60, total_minutes), df.columns.get_loc('meal_event')] = 1

        if np.random.rand() < person_profile['exercise_freq_prob']:
            exercise_start_time = int(np.random.normal(18.5, 1.0) * 60)
            event_start = day_start_idx + exercise_start_time
            if event_start < total_minutes: df.iloc[event_start : min(event_start + 60, total_minutes), df.columns.get_loc('exercise_event')] = 1

    minute_of_day_series = df.index.hour * 60 + df.index.minute
    df['heart_rate'] = 65 - 10 * df['is_sleeping'] - 8 * np.cos(2 * np.pi * (minute_of_day_series - 12*60) / DAY_MINUTES)
    df['hrv'] = 55 + 25 * df['is_sleeping'] + 10 * np.cos(2 * np.pi * (minute_of_day_series - 4*60) / DAY_MINUTES)
    df['respiration_rate'] = 16 - 3 * df['is_sleeping']
    df['skin_temp'] = 33.5 + 1.5 * np.sin(2 * np.pi * (minute_of_day_series - 20*60) / DAY_MINUTES)

    for event_type, event_col in [('meal', 'meal_event'), ('exercise', 'exercise_event')]:
        for start_idx in np.where(df[event_col].diff() == 1)[0]:
            duration = 60
            end_idx = start_idx + duration
            hr_peak, hrv_dip, resp_peak, peak_time, decay_rate = (8, -10, 2, start_idx + 45, 40) if event_type == 'meal' else (45, -30, 8, start_idx + 15, 30)
            time_indices = np.arange(start_idx, end_idx + 3*decay_rate)
            time_indices = time_indices[time_indices < total_minutes]
            effect = np.exp(-0.5 * ((time_indices - peak_time) / decay_rate)**2)
            df.iloc[time_indices, df.columns.get_loc('heart_rate')] += hr_peak * effect
            df.iloc[time_indices, df.columns.get_loc('hrv')] += hrv_dip * effect
            df.iloc[time_indices, df.columns.get_loc('respiration_rate')] += resp_peak * effect

    blackout_prob = 0.3
    is_blackout_night = (df['is_sleeping'] == 1) & (np.random.rand() < blackout_prob)
    light_penetration = np.where(is_blackout_night, 0.001, 0.05)
    df['perceived_lux'] = df['lux'] * np.where(df['is_sleeping'] == 1, light_penetration, 1.0)
    df['integrated_lux'] = df['perceived_lux'].rolling(window=30, min_periods=1).mean()
    df['ambient_temp'] = 22 + 5 * np.sin(2 * np.pi * (minute_of_day_series - 15*60) / DAY_MINUTES)

    for col, std in [('heart_rate', 1.5), ('hrv', 4), ('respiration_rate', 0.75), ('skin_temp', 0.15)]:
        df[col] += np.random.normal(0, std, total_minutes)

    df.index.name = 'timestamp'
    return df

# =============================================================================
# 4-1. 안정적인 기준 궤도 생성 함수
# =============================================================================
def get_stable_limit_cycle(params, output_dir, force_recalculate=False):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    cache_path = os.path.join(output_dir, "stable_limit_cycle.npy")
    if os.path.exists(cache_path) and not force_recalculate: return np.load(cache_path)

    print("안정 궤도를 새로 계산합니다 (최초 1회 실행)...")
    standard_sleep, standard_light = np.zeros(DAY_MINUTES), np.zeros(DAY_MINUTES)
    standard_sleep[0:8*60] = 1
    standard_light[8*60:10*60], standard_light[10*60:18*60], standard_light[18*60:22*60], standard_light[22*60:24*60] = 150, 400, 150, 50
    light_func_spin_up = lambda t: standard_light[int((t % 24) * 60)]
    sleep_func_spin_up = lambda t: standard_sleep[int((t % 24) * 60)]
    sol_spin_up = solve_ivp(fun=lco_model_ode, t_span=[0, 10 * 24], y0=[1.0, 0.0, 0.5], method='BDF', args=(params, light_func_spin_up, sleep_func_spin_up), dense_output=True, rtol=1e-6, atol=1e-9)
    if not sol_spin_up.success: raise RuntimeError(f"안정 궤도 계산 실패: {sol_spin_up.message}")

    limit_cycle_map = sol_spin_up.sol(np.arange(9 * 24, 10 * 24, 1.0/60.0)).T
    np.save(cache_path, limit_cycle_map)
    print(f"새로운 안정 궤도를 저장했습니다: {cache_path}")
    return limit_cycle_map

def generate_daily_baseline_trajectories(df, params, output_dir):
    print("--- [1단계] 일일 재조정 기준 궤도 생성 시작 ---")
    total_minutes, num_days = len(df), len(df) // DAY_MINUTES
    limit_cycle_map = get_stable_limit_cycle(params, output_dir)
    map_phases = np.arctan2(limit_cycle_map[:, 1], limit_cycle_map[:, 0])
    final_trajectory = np.zeros((total_minutes, 3))
    theoretical_anchor_minute, last_known_anchor_minute = 4 * 60, -1
    t_eval_day_hours = np.arange(DAY_MINUTES) / 60.0

    for day in tqdm(range(num_days), desc="일일 기준 궤도 생성 중"):
        day_start_idx, day_end_idx = day * DAY_MINUTES, (day + 1) * DAY_MINUTES
        day_df = df.iloc[day_start_idx:day_end_idx]
        day_hr, day_sleep = day_df['heart_rate'].values, day_df['is_sleeping'].values
        sleep_indices = np.where(day_sleep > 0.5)[0]

        current_anchor_minute = sleep_indices[np.argmin(day_hr[sleep_indices])] if len(sleep_indices) > 0 else -1
        if current_anchor_minute != -1: last_known_anchor_minute = current_anchor_minute
        anchor_to_use = last_known_anchor_minute if last_known_anchor_minute != -1 else theoretical_anchor_minute

        cbt_nadir_minute_in_day = (anchor_to_use + 120) % DAY_MINUTES
        initial_phase_at_midnight = (-170.7 * np.pi / 180.0) - (cbt_nadir_minute_in_day * (2 * np.pi) / DAY_MINUTES)
        initial_phase_at_midnight = (initial_phase_at_midnight + np.pi) % (2 * np.pi) - np.pi
        y0 = limit_cycle_map[np.argmin(np.abs((map_phases - initial_phase_at_midnight + np.pi) % (2 * np.pi) - np.pi))]

        day_df_smoothed = pd.DataFrame(index=day_df.index)
        day_df_smoothed['lux_smoothed'] = day_df['integrated_lux'].rolling(window=15, min_periods=1, center=True).mean()
        day_df_smoothed['sleep_smoothed'] = day_df['is_sleeping'].rolling(window=15, min_periods=1, center=True).mean()
        light_func = interp1d(t_eval_day_hours, day_df_smoothed['lux_smoothed'].values, kind='linear', fill_value="extrapolate")
        sleep_func = interp1d(t_eval_day_hours, day_df_smoothed['sleep_smoothed'].values, kind='linear', fill_value="extrapolate")

        sol_day = solve_ivp(fun=lco_model_ode, t_span=[0, 24], y0=y0, method='BDF', jac=lco_model_jacobian, args=(params, light_func, sleep_func), dense_output=True, t_eval=t_eval_day_hours, rtol=1e-5, atol=1e-8)

        if sol_day.success and sol_day.y.shape[1] == DAY_MINUTES:
            final_trajectory[day_start_idx:day_end_idx, :] = sol_day.y.T
        elif day > 0:
            print(f"경고: Day {day+1} 시뮬레이션 실패. 이전 날 궤도를 복사합니다.")
            final_trajectory[day_start_idx:day_end_idx, :] = final_trajectory[(day-1)*DAY_MINUTES:day*DAY_MINUTES, :]

    if total_minutes > num_days * DAY_MINUTES and num_days > 0:
        remaining_start, remaining_len = num_days * DAY_MINUTES, total_minutes - num_days * DAY_MINUTES
        final_trajectory[remaining_start:, :] = final_trajectory[(num_days-1)*DAY_MINUTES : (num_days-1)*DAY_MINUTES+remaining_len, :]

    print("--- [1단계] 일일 재조정 기준 궤도 생성 완료 ---")
    return final_trajectory

# =============================================================================
# 5. 시각화 함수 (상세 리포트 기능 추가)
# =============================================================================
def find_all_markers(df_unscaled):
    print("--- 전체 데이터에 대한 생체 마커 탐색 시작 ---")
    df = df_unscaled.copy()
    df['corrected_skin_temp'] = df['skin_temp'] - df['ambient_temp']
    all_markers = {'onset': [], 'nadir': [], 'offset': []}
    num_days = len(df) // DAY_MINUTES
    for day in range(num_days):
        day_start, day_end = day * DAY_MINUTES, (day + 1) * DAY_MINUTES
        day_df = df.iloc[day_start:day_end]
        sleep_hr = day_df['heart_rate'][day_df['is_sleeping'] > 0.5]
        if not sleep_hr.empty: all_markers['nadir'].append(sleep_hr.idxmin())
        temp_series = day_df['corrected_skin_temp'].values
        if len(temp_series) > 40:
            temp_deriv_smoothed = moving_average_np(np.gradient(moving_average_np(temp_series, 30)), 10)
            offset_to_align = len(temp_series) - len(temp_deriv_smoothed)
            onset_candidates, offset_candidates = np.where(temp_deriv_smoothed > 0.001)[0] + offset_to_align, np.where(temp_deriv_smoothed < -0.001)[0] + offset_to_align
            if len(onset_candidates) > 0: all_markers['onset'].append(day_df.index[onset_candidates[0]])
            if len(offset_candidates) > 0: all_markers['offset'].append(day_df.index[offset_candidates[-1]])
    print("--- 생체 마커 탐색 완료 ---")
    return all_markers

def plot_full_data_report(df_raw, all_markers, output_dir):
    print("--- 전체 데이터 상세 리포트 생성 시작 ---")
    total_days, days_per_chunk = len(df_raw) // DAY_MINUTES, 3
    num_chunks = math.ceil(total_days / days_per_chunk)

    for i in range(num_chunks):
        start_day, end_day = i * days_per_chunk, min((i + 1) * days_per_chunk, total_days)
        chunk_df = df_raw.iloc[start_day * DAY_MINUTES : end_day * DAY_MINUTES]
        if chunk_df.empty: continue

        fig, axs = plt.subplots(4, 1, figsize=(20, 22), sharex=True)
        fig.suptitle(f'Biometric Data Report (Days {start_day + 1}-{end_day})', fontsize=20, y=0.95)

        ax1, ax1_twin = axs[0], axs[0].twinx()
        ax1.set_title('Heart Rate, Skin Temperature & Sleep', fontsize=14)
        ax1.set_ylabel('Heart Rate (bpm)', color='tab:red'); ax1.plot(chunk_df.index, chunk_df['heart_rate'], color='tab:red', label='Heart Rate', zorder=10); ax1.tick_params(axis='y', labelcolor='tab:red')
        ax1_twin.set_ylabel('Skin Temp (°C)', color='tab:purple'); ax1_twin.plot(chunk_df.index, chunk_df['skin_temp'], color='tab:purple', label='Skin Temp', zorder=10); ax1_twin.tick_params(axis='y', labelcolor='tab:purple')
        ax1.fill_between(chunk_df.index, ax1.get_ylim()[0], ax1.get_ylim()[1], where=chunk_df['is_sleeping'] > 0.5, color='gray', alpha=0.2, label='Sleep', zorder=0)

        ax2, ax2_twin = axs[1], axs[1].twinx()
        ax2.set_title('HRV, Respiration Rate & Exercise', fontsize=14)
        ax2.set_ylabel('HRV (ms)', color='tab:green'); ax2.plot(chunk_df.index, chunk_df['hrv'], color='tab:green', label='HRV', zorder=10); ax2.tick_params(axis='y', labelcolor='tab:green')
        ax2_twin.set_ylabel('Respiration (brpm)', color='tab:brown'); ax2_twin.plot(chunk_df.index, chunk_df['respiration_rate'], color='tab:brown', label='Respiration Rate', zorder=10); ax2_twin.tick_params(axis='y', labelcolor='tab:brown')
        ax2.fill_between(chunk_df.index, ax2.get_ylim()[0], ax2.get_ylim()[1], where=chunk_df['exercise_event'] > 0.5, color='orange', alpha=0.3, label='Exercise', zorder=0)

        ax3 = axs[2]; ax3.set_title('Light Exposure', fontsize=14); ax3.set_ylabel('Lux'); ax3.plot(chunk_df.index, chunk_df['lux'], color='orange', label='Lux'); ax3.set_yscale('log'); ax3.set_ylim(bottom=1)
        ax4 = axs[3]; ax4.set_title('Meal Events', fontsize=14); ax4.set_ylabel('Meal Event'); ax4.fill_between(chunk_df.index, 0, 1, where=chunk_df['meal_event'] > 0.5, color='skyblue', alpha=0.5, label='Meal'); ax4.set_yticks([0, 1])

        marker_colors = {'onset': 'green', 'nadir': 'blue', 'offset': 'purple'}
        for ax in axs:
            ax.grid(True, which='major', linestyle='--', linewidth=0.5)
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%H:%M')); ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
            for marker_type, timestamps in all_markers.items():
                for ts in timestamps:
                    if chunk_df.index[0] <= ts < chunk_df.index[-1]: ax.axvline(x=ts, color=marker_colors[marker_type], linestyle='--', linewidth=1.5, label=f'_{marker_type}')

        lines1, labels1 = ax1.get_legend_handles_labels(); lines1_t, labels1_t = ax1_twin.get_legend_handles_labels(); ax1.legend(lines1 + lines1_t, labels1 + labels1_t, loc='upper left')
        lines2, labels2 = ax2.get_legend_handles_labels(); lines2_t, labels2_t = ax2_twin.get_legend_handles_labels(); ax2.legend(lines2 + lines2_t, labels2 + labels2_t, loc='upper left')
        ax3.legend(loc='upper left'); ax4.legend(loc='upper left')

        from matplotlib.lines import Line2D
        legend_elements = [Line2D([0], [0], color='green', lw=2, linestyle='--', label='Onset'), Line2D([0], [0], color='blue', lw=2, linestyle='--', label='Nadir'), Line2D([0], [0], color='purple', lw=2, linestyle='--', label='Offset')]
        fig.legend(handles=legend_elements, loc='upper right', fontsize=12)

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        save_path = os.path.join(output_dir, f'report_days_{start_day+1}-{end_day}.png')
        plt.savefig(save_path); plt.close(fig)
        # print(f"상세 리포트 그래프가 {save_path}에 저장되었습니다.")

# =============================================================================
# 6. 학습, 평가, 예측 파이프라인
# =============================================================================
def train_and_evaluate(train_tfrecord, val_tfrecord, num_train_samples, num_val_samples, config):
    print("\n--- 최종 모델 학습 및 검증 시작 ---")

    main_cnn_other_keys = [ 'input_lux', 'input_sleep', 'input_body1', 'input_body2', 'input_zeit1', 'input_zeit2', 'input_zeit3', ]
    fourier_corr_keys = [ 'corr_input_lux', 'corr_input_sleep', 'corr_input_body1', 'corr_input_body2', 'corr_input_zeit1', 'corr_input_zeit2', 'corr_input_zeit3', ]
    other_keys = ['baseline_inputs', 'full_inputs_for_anchor', 'past_info_indices', 'label', 'time_offset']
    feature_spec = {key: tf.io.FixedLenFeature([], tf.string) for key in main_cnn_other_keys + fourier_corr_keys + other_keys}

    def _parse_function(example_proto):
        parsed = tf.io.parse_single_example(example_proto, feature_spec)

        inputs = {}
        for key in main_cnn_other_keys: inputs[key] = tf.io.parse_tensor(parsed[key], out_type=tf.float32)
        for key in fourier_corr_keys: inputs[key] = tf.io.parse_tensor(parsed[key], out_type=tf.float32)

        inputs['baseline_inputs'] = tf.io.parse_tensor(parsed['baseline_inputs'], out_type=tf.float32)
        inputs['past_info_indices'] = tf.io.parse_tensor(parsed['past_info_indices'], out_type=tf.int64)
        inputs['full_inputs_for_anchor'] = tf.io.parse_tensor(parsed['full_inputs_for_anchor'], out_type=tf.float32)
        inputs['time_offset'] = tf.io.parse_tensor(parsed['time_offset'], out_type=tf.int32)

        label = tf.io.parse_tensor(parsed['label'], out_type=tf.float32)
        return inputs, label

    def make_dataset(file_path):
        return tf.data.TFRecordDataset(file_path, num_parallel_reads=tf.data.AUTOTUNE).map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE).shuffle(256).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)

    train_dataset, val_dataset = make_dataset(train_tfrecord), make_dataset(val_tfrecord)

    lco_feature_extractor, other_feature_extractor = build_main_feature_extractors(config['input_seq_len'], config['d_model'])
    fourier_correction_model = build_fourier_correction_model(config['lookback_days'] * DAY_MINUTES, config['num_harmonics'], config['lstm_units'])
    main_predictor_config = {k: v for k, v in config.items() if k in ['num_layers', 'd_model', 'num_heads', 'dff', 'rate']}
    main_predictor = MainPredictor(prediction_horizon=config['pred_horizon'], **main_predictor_config)
    integrated_model = IntegratedModel(lco_feature_extractor, other_feature_extractor, fourier_correction_model, main_predictor, config)

    target_scaler = joblib.load(os.path.join(OUTPUT_DIR, 'target_scaler.gz'))
    target_scaler_mean, target_scaler_scale = tf.constant(target_scaler.mean_[0], dtype=tf.float32), tf.constant(target_scaler.scale_[0], dtype=tf.float32)
    mse_loss_fn, optimizer = tf.keras.losses.MeanSquaredError(), Adam(learning_rate=config['learning_rate'])

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            y_pred = integrated_model(x, training=True)
            main_loss = mse_loss_fn((y - target_scaler_mean) / target_scaler_scale, y_pred)
            total_loss = main_loss + sum(integrated_model.losses)
        grads = tape.gradient(total_loss, integrated_model.trainable_variables)
        optimizer.apply_gradients(zip(tf.clip_by_global_norm(grads, 1.0)[0], integrated_model.trainable_variables))
        return total_loss

    @tf.function
    def val_step(x, y):
        y_pred = integrated_model(x, training=False)
        return mse_loss_fn((y - target_scaler_mean) / target_scaler_scale, y_pred)

    history, best_val_loss = {'train_loss': [], 'val_loss': []}, float('inf')
    model_save_path = os.path.join(OUTPUT_DIR, "best_model.weights.h5")

    for epoch in range(config['epochs']):
        print(f"\nEpoch {epoch + 1}/{config['epochs']}")
        progbar = tf.keras.utils.Progbar(num_train_samples // config['batch_size'], stateful_metrics=['train_loss'])
        epoch_train_losses = []
        for i, (x, y) in enumerate(train_dataset):
            loss = train_step(x, y)
            progbar.update(i + 1, values=[('train_loss', loss)])
            epoch_train_losses.append(loss)
        history['train_loss'].append(np.mean(epoch_train_losses))

        val_losses = [val_step(x, y) for x, y in val_dataset]
        val_loss = tf.reduce_mean(val_losses)
        history['val_loss'].append(val_loss.numpy())
        print(f"\nValidation Loss (scaled): {val_loss.numpy():.4f}")

        if val_loss < best_val_loss:
            print(f"Validation loss improved. Saving model weights to {model_save_path}")
            best_val_loss = val_loss
            integrated_model.save_weights(model_save_path)

    print("\n--- 학습 및 검증 완료 ---")
    return integrated_model, val_dataset, history

def predict_future_hr(model, data_slice_unscaled, data_slice_scaled, config):
    print("\n--- 미래 심박수 예측 시작 ---")

    main_cnn_other_cols = { 'input_lux': ['lux'], 'input_sleep': ['is_sleeping'], 'input_body1': ['heart_rate', 'hrv', 'respiration_rate'], 'input_body2': ['skin_temp'], 'input_zeit1': ['meal_event'], 'input_zeit2': ['exercise_event'], 'input_zeit3': ['ambient_temp'], }
    fourier_corr_cols = { 'corr_input_lux': ['lux'], 'corr_input_sleep': ['is_sleeping'], 'corr_input_body1': ['heart_rate', 'hrv', 'respiration_rate'], 'corr_input_body2': ['skin_temp'], 'corr_input_zeit1': ['meal_event'], 'corr_input_zeit2': ['exercise_event'], 'corr_input_zeit3': ['ambient_temp'], }
    anchor_loss_cols = ['heart_rate', 'is_sleeping']

    # 예측에서는 단일 샘플이므로, 시작점을 0으로 가정 (오프셋 0)
    time_offset = 0
    num_main_seq_days = config['input_seq_len'] // DAY_MINUTES
    total_days_needed = config['lookback_days'] + num_main_seq_days

    full_slice_scaled = data_slice_scaled.iloc[:total_days_needed * DAY_MINUTES]

    model_input = {}
    for key, cols in main_cnn_other_cols.items(): model_input[key] = tf.constant(full_slice_scaled[cols].values[np.newaxis, ...], dtype=tf.float32)
    for key, cols in fourier_corr_cols.items(): model_input[key] = tf.constant(full_slice_scaled[cols].values[np.newaxis, ...], dtype=tf.float32)

    model_input['baseline_inputs'] = tf.constant(full_slice_scaled[['x_base', 'xc_base']].values[np.newaxis, ...], dtype=tf.float32)
    model_input['time_offset'] = tf.constant([[time_offset]], dtype=tf.int32)

    anchor_data = data_slice_unscaled.iloc[config['input_seq_len'] - DAY_MINUTES : config['input_seq_len']]
    model_input['full_inputs_for_anchor'] = tf.constant(anchor_data[anchor_loss_cols].values[np.newaxis, ...], dtype=tf.float32)

    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    # [수정된 호출] 예측 시에도 개선된 함수를 사용하여 past_info 인덱스 생성
    model_input['past_info_indices'] = tf.constant(
        generate_past_info_indices(
            # [수정] iloc을 사용해 정확히 10일치 데이터만 전달
            full_unscaled_df=data_slice_unscaled.iloc[:total_days_needed * DAY_MINUTES],
            main_seq_start_in_full_df=time_offset, # 예측 시에는 0
            config=config
        )[np.newaxis, ...],
        dtype=tf.int64
    )
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

    target_scaler = joblib.load(os.path.join(OUTPUT_DIR, 'target_scaler.gz'))
    prediction_original = target_scaler.inverse_transform(model.predict(model_input))
    print("--- 예측 완료 ---")
    return prediction_original.flatten()

def plot_learning_curve(history, output_dir):
    plt.figure(figsize=(10, 6)); plt.plot(history['train_loss'], label='Training Loss'); plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Learning Curve'); plt.xlabel('Epoch'); plt.ylabel('Loss (Scaled MSE)'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(output_dir, "learning_curve.png")); plt.close()
    print(f"학습 곡선 그래프가 {os.path.join(output_dir, 'learning_curve.png')}에 저장되었습니다.")

def plot_performance_summary(actual, predicted, output_dir):
    mae, rmse = np.mean(np.abs(actual - predicted)), np.sqrt(np.mean((actual - predicted)**2))
    plt.figure(figsize=(8, 8)); plt.scatter(actual, predicted, alpha=0.5); plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2, label='Ideal')
    plt.title(f'Prediction vs Actual\nMAE: {mae:.2f}, RMSE: {rmse:.2f}'); plt.xlabel('Actual Heart Rate (bpm)'); plt.ylabel('Predicted Heart Rate (bpm)'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(output_dir, "performance_summary.png")); plt.close()
    print(f"성능 요약 그래프가 {os.path.join(output_dir, 'performance_summary.png')}에 저장되었습니다.")

# =============================================================================
# 7. 메인 실행 블록 및 모듈화된 함수
# =============================================================================
def generate_and_visualize_data(config, person_profile):
    print("--- 데이터 생성 및 시각화 시작 ---")
    num_main_seq_days = config['input_seq_len'] // DAY_MINUTES
    total_days_to_generate = config.get('data_duration_days', DATA_DURATION_DAYS) + config['lookback_days'] + num_main_seq_days + (config['pred_horizon'] // DAY_MINUTES) + 2

    df_full = generate_realistic_biometric_data(OUTPUT_DIR, total_days_to_generate, person_profile)
    canonical_columns = ['heart_rate', 'hrv', 'respiration_rate', 'skin_temp', 'ambient_temp', 'lux', 'is_sleeping', 'meal_event', 'exercise_event', 'integrated_lux']
    df_raw = df_full[canonical_columns]

    baseline_trajectory = generate_daily_baseline_trajectories(df_full, PARAMS, OUTPUT_DIR)
    df = df_raw.copy(); df[['x_base', 'xc_base', 'n_base']] = baseline_trajectory
    df['x'], df['xc'] = 0.0, 0.0
    for col in df.columns:
        if df[col].dtype == 'object' or pd.api.types.is_integer_dtype(df[col]): df[col] = df[col].astype(float)

    all_markers = find_all_markers(df_raw.iloc[:config.get('data_duration_days', DATA_DURATION_DAYS) * DAY_MINUTES])
    plot_full_data_report(df_raw.iloc[:config.get('data_duration_days', DATA_DURATION_DAYS) * DAY_MINUTES], all_markers, OUTPUT_DIR)
    print("--- 데이터 생성 및 시각화 완료 ---")
    return df_raw, df

def run_training_pipeline(config, person_profile):
    print("--- 'train' 모드 시작: 전체 파이프라인을 실행합니다. ---")
    df_raw, df = generate_and_visualize_data(config, person_profile)
    print("\n--- 데이터 분할 및 스케일링 시작 ---")

    num_main_seq_days = config['input_seq_len'] // DAY_MINUTES
    total_days_needed_for_sample = config['lookback_days'] + num_main_seq_days
    max_len_for_sample = total_days_needed_for_sample * DAY_MINUTES + config['pred_horizon']

    num_samples = len(df) - max_len_for_sample
    all_indices = np.arange(num_samples); np.random.shuffle(all_indices)
    train_end, val_end = int(num_samples * TRAIN_RATIO), int(num_samples * (TRAIN_RATIO + VALIDATION_RATIO))
    train_indices, val_indices = all_indices[:train_end], all_indices[train_end:val_end]

    feature_cols = [c for c in df.columns if c not in ['x', 'xc']]
    feature_scaler, target_scaler = StandardScaler(), StandardScaler()
    feature_scaler.fit(df.iloc[:train_indices[-1] if len(train_indices) > 0 else 0][feature_cols])
    target_scaler.fit(df.iloc[:train_indices[-1] if len(train_indices) > 0 else 0][['heart_rate']])
    df_scaled = df.copy(); df_scaled[feature_cols] = feature_scaler.transform(df[feature_cols])
    joblib.dump(feature_scaler, os.path.join(OUTPUT_DIR, 'feature_scaler.gz'))
    joblib.dump(target_scaler, os.path.join(OUTPUT_DIR, 'target_scaler.gz'))
    print("스케일러 학습 및 저장 완료.")

    train_tfrecord_path, val_tfrecord_path = os.path.join(OUTPUT_DIR, "train.tfrecord"), os.path.join(OUTPUT_DIR, "val.tfrecord")
    create_tfrecords(df, df_scaled, train_indices, config, train_tfrecord_path)
    create_tfrecords(df, df_scaled, val_indices, config, val_tfrecord_path)

    _, val_dataset_for_build, history = train_and_evaluate(train_tfrecord_path, val_tfrecord_path, len(train_indices), len(val_indices), config)
    plot_learning_curve(history, OUTPUT_DIR)

    run_prediction_only(config, person_profile, df, df_raw, val_dataset_for_build)
    visualize_architecture_only(config)
    print("\n\n" + "="*59 + "\n    생체리듬 예측 AI 전체 파이프라인 실행 완료 (v0.16.1)   \n" + "="*59)

def run_data_generation_only(config, person_profile):
    print("\n--- 'generate_data' 모드 시작: 데이터 생성 및 그래프 출력만 수행합니다. ---")
    generate_and_visualize_data(config, person_profile)
    print("\n" + "="*59 + "\n    데이터 생성 및 시각화 완료 (generate_data 모드)   \n" + "="*59)

def run_prediction_only(config, person_profile, df=None, df_raw=None, val_dataset_for_build=None):
    print("\n--- 'predict' 모드 시작: 저장된 모델로 예측을 수행합니다. ---")
    model_weights_path = os.path.join(OUTPUT_DIR, "best_model.weights.h5")
    if not os.path.exists(model_weights_path):
        print(f"오류: 모델 가중치 파일({model_weights_path})을 찾을 수 없습니다. 'train' 모드를 먼저 실행해주세요."); return

    lco_feat, other_feat = build_main_feature_extractors(config['input_seq_len'], config['d_model'])
    fourier_model = build_fourier_correction_model(config['lookback_days'] * DAY_MINUTES, config['num_harmonics'], config['lstm_units'])
    predictor_config = {k: v for k, v in config.items() if k in ['num_layers', 'd_model', 'num_heads', 'dff', 'rate']}
    predictor = MainPredictor(prediction_horizon=config['pred_horizon'], **predictor_config)
    loaded_model = IntegratedModel(lco_feat, other_feat, fourier_model, predictor, config)

    if val_dataset_for_build is None:
        print("경고: 검증 데이터셋이 없어 더미 데이터로 모델을 빌드합니다.")
        dummy_batch_size = 2
        dummy_x = {}
        num_main_seq_days = config['input_seq_len'] // DAY_MINUTES
        total_days_needed = config['lookback_days'] + num_main_seq_days
        total_minutes_needed = total_days_needed * DAY_MINUTES

        main_cnn_other_keys = [ 'input_lux', 'input_sleep', 'input_body1', 'input_body2', 'input_zeit1', 'input_zeit2', 'input_zeit3', ]
        main_cnn_other_shapes = [1, 1, 3, 1, 1, 1, 1]
        for key, shape in zip(main_cnn_other_keys, main_cnn_other_shapes):
            dummy_x[key] = tf.zeros((dummy_batch_size, total_minutes_needed, shape))

        fourier_corr_keys = [ 'corr_input_lux', 'corr_input_sleep', 'corr_input_body1', 'corr_input_body2', 'corr_input_zeit1', 'corr_input_zeit2', 'corr_input_zeit3', ]
        for key, shape in zip(fourier_corr_keys, main_cnn_other_shapes):
            dummy_x[key] = tf.zeros((dummy_batch_size, total_minutes_needed, shape))

        dummy_x['baseline_inputs'] = tf.zeros((dummy_batch_size, total_minutes_needed, 2))
        dummy_x['past_info_indices'] = tf.zeros((dummy_batch_size, config['num_markers_to_keep'] * 3), dtype=tf.int64)
        dummy_x['full_inputs_for_anchor'] = tf.zeros((dummy_batch_size, DAY_MINUTES, 2))
        dummy_x['time_offset'] = tf.zeros((dummy_batch_size, 1), dtype=tf.int32)
    else: dummy_x, _ = next(iter(val_dataset_for_build))

    loaded_model(dummy_x, training=False); loaded_model.load_weights(model_weights_path)
    print("모델 가중치 로드 완료.")

    if df is None or df_raw is None: df_raw, df = generate_and_visualize_data(config, person_profile)
    feature_scaler = joblib.load(os.path.join(OUTPUT_DIR, 'feature_scaler.gz'))
    df_scaled = df.copy(); df_scaled[[c for c in df.columns if c not in ['x', 'xc']]] = feature_scaler.transform(df[[c for c in df.columns if c not in ['x', 'xc']]])

    num_main_seq_days = config['input_seq_len'] // DAY_MINUTES
    total_days_needed = config['lookback_days'] + num_main_seq_days
    max_len = total_days_needed * DAY_MINUTES + config['pred_horizon']

    test_slice_start = len(df) - max_len
    test_data_slice_unscaled = df.iloc[test_slice_start : test_slice_start + max_len]
    test_data_slice_scaled = df_scaled.iloc[test_slice_start : test_slice_start + max_len]

    predicted_hr = predict_future_hr(loaded_model, test_data_slice_unscaled, test_data_slice_scaled, config)

    actual_hr_df = df_raw.iloc[-config['pred_horizon']:]

    plt.figure(figsize=(15, 7)); plt.plot(actual_hr_df.index, actual_hr_df['heart_rate'].values, label='Actual Heart Rate', color='blue'); plt.plot(actual_hr_df.index, predicted_hr, label='Predicted Heart Rate', color='red', linestyle='--')
    plt.title('Heart Rate Prediction vs Actual (Time Series)'); plt.xlabel('Time'); plt.ylabel('Heart Rate (bpm)'); plt.legend(); plt.xticks(rotation=45); plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "prediction_vs_actual.png")); plt.close()
    print(f"\n예측 결과 그래프가 {os.path.join(output_dir, 'prediction_vs_actual.png')}에 저장되었습니다.")
    plot_performance_summary(actual_hr_df['heart_rate'].values, predicted_hr, OUTPUT_DIR)

def visualize_architecture_only(config):
    print("\n--- 'visualize_arch' 모드 시작: 모델 구조도를 생성합니다. ---")
    if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
    try:
        lco_feat, other_feat = build_main_feature_extractors(config['input_seq_len'], config['d_model'])
        fourier_model = build_fourier_correction_model(config['lookback_days'] * DAY_MINUTES, config['num_harmonics'], config['lstm_units'])
        tf.keras.utils.plot_model(lco_feat, to_file=os.path.join(OUTPUT_DIR, 'lco_feature_extractor.png'), show_shapes=True)
        tf.keras.utils.plot_model(other_feat, to_file=os.path.join(OUTPUT_DIR, 'other_feature_extractor.png'), show_shapes=True)
        tf.keras.utils.plot_model(fourier_model, to_file=os.path.join(OUTPUT_DIR, 'fourier_correction_model.png'), show_shapes=True)
        print(f"모델 구조 이미지가 {OUTPUT_DIR}에 저장되었습니다.")
    except Exception as e:
        print(f"모델 시각화 중 오류 발생: {e}\npydot과 graphviz가 설치되어 있는지 확인해주세요. (pip install pydot graphviz)")

if __name__ == '__main__':
    EXECUTION_MODE = 'train'

    person_profile_office_worker = {
        'wake_up_weekday_mean': 7.5, 'wake_up_weekday_std': 0.75,
        'wake_up_weekend_mean': 9.5, 'wake_up_weekend_std': 1.5,
        'sleep_duration_mean': 7.0, 'exercise_freq_prob': 0.4,
    }

    config = {
        'batch_size': BATCH_SIZE, 'input_seq_len': INPUT_SEQUENCE_LENGTH,
        'pred_horizon': PREDICTION_HORIZON, 'lookback_days': PHASE_CORRECTION_LOOKBACK_DAYS,
        'day_minutes': DAY_MINUTES, 'num_markers_to_keep': NUM_MARKERS_TO_KEEP,
        'lambda_reg': LAMBDA_REG, 'lambda_cont': LAMBDA_CONT, 'lambda_anchor': LAMBDA_ANCHOR,
        'num_layers': NUM_LAYERS, 'd_model': D_MODEL, 'num_heads': NUM_HEADS, 'dff': DFF, 'rate': DROPOUT_RATE,
        'epochs': EPOCHS, 'learning_rate': LEARNING_RATE, 'num_harmonics': NUM_FOURIER_HARMONICS,
        'data_duration_days': DATA_DURATION_DAYS,
        'lstm_units': LSTM_UNITS,
    }

    if EXECUTION_MODE == 'train':
        run_training_pipeline(config, person_profile_office_worker)
    elif EXECUTION_MODE == 'predict':
        run_prediction_only(config, person_profile_office_worker)
    elif EXECUTION_MODE == 'visualize_arch':
        visualize_architecture_only(config)
    elif EXECUTION_MODE == 'generate_data':
        run_data_generation_only(config, person_profile_office_worker)
    else:
        print(f"알 수 없는 모드입니다: {EXECUTION_MODE}. 'train', 'predict', 'visualize_arch', 'generate_data' 중에서 선택해주세요.")


--- 'train' 모드 시작: 전체 파이프라인을 실행합니다. ---
--- 데이터 생성 및 시각화 시작 ---
--- [1단계] 일일 재조정 기준 궤도 생성 시작 ---


일일 기준 궤도 생성 중: 100%|██████████| 42/42 [00:06<00:00,  6.10it/s]


--- [1단계] 일일 재조정 기준 궤도 생성 완료 ---
--- 전체 데이터에 대한 생체 마커 탐색 시작 ---
--- 생체 마커 탐색 완료 ---
--- 전체 데이터 상세 리포트 생성 시작 ---
--- 데이터 생성 및 시각화 완료 ---

--- 데이터 분할 및 스케일링 시작 ---
스케일러 학습 및 저장 완료.
--- TFRecord 파일 생성 시작: final_biometric_pipeline_output_v0.16.1_PastInfoFix/train.tfrecord ---


train.tfrecord 생성 중: 100%|██████████| 36816/36816 [17:39<00:00, 34.75it/s]


--- TFRecord 파일 생성 완료: final_biometric_pipeline_output_v0.16.1_PastInfoFix/train.tfrecord ---
--- TFRecord 파일 생성 시작: final_biometric_pipeline_output_v0.16.1_PastInfoFix/val.tfrecord ---


val.tfrecord 생성 중: 100%|██████████| 4602/4602 [02:13<00:00, 34.44it/s]


--- TFRecord 파일 생성 완료: final_biometric_pipeline_output_v0.16.1_PastInfoFix/val.tfrecord ---

--- 최종 모델 학습 및 검증 시작 ---

Epoch 1/15
